defining a base class for layers


In [59]:
import numpy as np
from Activation_Functions.Layer import Layer

In [60]:
#linear layer 
class Linear(Layer):
    def __init__ (self, input_dim, output_dim):
        super().__init__()
        self.w = 0.1 * np.random.rand(input_dim, output_dim)
        self.b = np.zeros((1, output_dim))
        self.dw = np.zeros_like(self.w)
        self.db = np.zeros_like(self.b)
    def forward(self, input):
        self.input  = input
        self.output= np.dot(input, self.w) + self.b
        return self.output
    def backward(self, up_gradient: np.ndarray) -> np.ndarray:
        """Backpropagate the gradients through this layer."""
        #dw = dL/dw
        self.dw = np.dot(self.input.T, up_gradient)  
        self.db = np.sum(up_gradient, axis=0, keepdims=True)
        down_grad = np.dot(up_gradient, self.w.T)
        return down_grad
    def step(self, learning_rate: float) -> None:
        """Update the weights and biases using the gradients."""
        self.w -= learning_rate * self.dw
        self.b -= learning_rate * self.db

In [61]:
#testing the linearilty
np.random.seed(0)  
linear_layer = Linear(3, 2)
input = np.array([[1.0, 2.0, 3.0]]) 
output = linear_layer(input)
print("Input:\n", input)
print("Weights:\n", linear_layer.w)
print("Biases:\n", linear_layer.b)
print("Output:\n", output)

Input:
 [[1. 2. 3.]]
Weights:
 [[0.05488135 0.07151894]
 [0.06027634 0.05448832]
 [0.04236548 0.06458941]]
Biases:
 [[0. 0.]]
Output:
 [[0.30253047 0.37426381]]


In [70]:
class MultiLayerPerceptron:
    def __init__(self, layers, loss_function, learning_rate):
        self.layers = layers
        self.loss_function = loss_function
        self.learning_rate = learning_rate
    def __call__(self, input):
        #forward passing the model
        return self.forward(input)
    def forward (self, input):
        for layer in self.layers:
            input = layer.forward(input)
        return input
    def loss(self, prediction, target):
        return self.loss_function(prediction, target)
    def backward(self):
        up_gradient = self.loss_function.backward()
        for layer in reversed(self.layers):
            up_gradient = layer.backward(up_gradient)
    def update(self):
        for layer in self.layers:
            layer.step(self.learning_rate)
    def train(self, x_train, y_train, epochs, batch_size):
        losses = np.empty(epochs)
        for epoch in ((epochs)):
            running_loss = 0.0
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                prediction = self.forward(x_batch)
                running_loss += self.loss(prediction, y_batch) * batch_size
                self.backward()
                self.update()
            running_loss /= len(x_train)
            print(f'loss is equal to {running_loss: .2f}')
            losses[epoch] = running_loss
        return losses

In [74]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic dataset
n_samples = 100

# Feature 1: Study hours per week (between 1 and 12)
study_hours = np.random.uniform(1, 12, n_samples)

# Feature 2: Previous test score (between 50 and 100)
previous_score = np.random.uniform(50, 100, n_samples)

# Creating a formula that combines both features to determine probability of passing
# Higher study hours and previous scores increase chance of passing
probability = 1 / (1 + np.exp(-(0.7 * study_hours + 0.05 * previous_score - 8)))

# Generate binary outcome (pass/fail) based on probability
# 1 = pass, 0 = fail
passed = np.random.binomial(1, probability)

# Create feature matrix X and target vector y
X = np.column_stack((study_hours, previous_score))
y = passed

# Split data into training (70%) and testing (30%) sets
# First, create indices for the split
n_train = int(0.7 * n_samples)
indices = np.random.permutation(n_samples)
train_idx, test_idx = indices[:n_train], indices[n_train:]

# Create train/test splits
X_train = X[train_idx]
y_train = y[train_idx]
X_test = X[test_idx]
y_test = y[test_idx]

# Print dataset sizes
print(f"Total dataset: {n_samples} samples")
print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# Print first 5 samples from training set
print("\nFirst 5 training samples (X_train, y_train):")
for i in range(min(5, len(X_train))):
    print(f"Features: {X_train[i]}, Target: {y_train[i]}")

# Print first 5 samples from test set
print("\nFirst 5 test samples (X_test, y_test):")
for i in range(min(5, len(X_test))):
    print(f"Features: {X_test[i]}, Target: {y_test[i]}")

# Basic statistics for train/test sets
print("\nTraining set statistics:")
print(f"Pass rate: {np.mean(y_train):.2f}")
print("\nTest set statistics:")
print(f"Pass rate: {np.mean(y_test):.2f}")

# Optionally save to NumPy files
# np.save('X_train_logistic.npy', X_train)
# np.save('y_train_logistic.npy', y_train)
# np.save('X_test_logistic.npy', X_test)
# np.save('y_test_logistic.npy', y_test)

Total dataset: 100 samples
Training set: 70 samples
Test set: 30 samples

First 5 training samples (X_train, y_train):
Features: [ 1.63891973 70.51914615], Target: 0
Features: [ 6.01676983 61.39675813], Target: 0
Features: [ 2.71593972 62.46461146], Target: 0
Features: [ 2.07439325 85.15094794], Target: 0
Features: [ 4.42080554 62.89708139], Target: 0

First 5 test samples (X_test, y_test):
Features: [ 1.82005708 52.03875708], Target: 0
Features: [10.75934017 76.48252892], Target: 1
Features: [ 4.92428659 76.7887342 ], Target: 0
Features: [ 5.27545019 68.38915664], Target: 0
Features: [ 1.27961039 94.35432121], Target: 0

Training set statistics:
Pass rate: 0.53

Test set statistics:
Pass rate: 0.47


In [ ]:
X_train.shape
from Activation_Functions.Sigmoid import Sigmoid
from Loss_Functions.MSE import MSE

ImportError: cannot import name 'MSE' from 'Loss_Functions.MSE' (/home/majid/Desktop/WebDev/Machine_Learning_From_Scratch/NN/../Loss_Functions/MSE.py)

In [79]:
layers=[Linear(40, 2), Sigmoid()]

In [81]:
model = MultiLayerPerceptron(layers, MSE(), learning_rate=0.01)


NameError: name 'MSE' is not defined